# Trabalho Prático 1
## Computação Gráfica

Deiziane Natani da Silva  
2015121980  

Implementação do Flat shading, Gouraud Shading e Phong Shading.

In [1]:
import glfw
import pyrr
import numpy
from OpenGL.GL import *
import OpenGL.GL.shaders


Shaders descrevem como processar dados no pipeline gráfico, ou seja, são um conjunto de instruções.

No código estão dois tipos de shader:

- **Vertex Shader**:
   - Como o nome indica, é o responsável pela manipulação dos vértices de um objeto tridimensional. O vertex shader processa cada ponto/vértice que você passa e retorna valores de posição para esse ponto.
   
   
- **Fragment Shader**:
   - Trabalha num pixel, ou um fragmento. É onde a profundidade, cor e todos os outros parâmetros dos gráficos são processados de acordo com as condições ambientais e de iluminação especificada


In [17]:
vertex_shader = '''

#version 330

uniform mat4 model_transform;
uniform mat4 camera;
uniform mat4 projection;
uniform vec3 camera_pos_input;

uniform float albedo_r;
uniform float albedo_g;
uniform float albedo_b;
uniform float ks;

in vec3 position;
in vec3 vertex_normal;

out vec3 view_position;
out vec3 pixel_position;
out vec3 vertex_color;
out vec3 camera_position;
out vec3 normal;
out vec3 light_source;
out vec4 ambient_light;

void main() {
    gl_Position = projection * camera * model_transform * vec4(position, 1.0);
    view_position = gl_Position.xyz;
    pixel_position = position;
    camera_position = camera_pos_input;
    
    if(vertex_normal == vec3(0.0, 0.0, 0.0)) {
        normal = normalize(position);
    }
    else {
        normal = normalize(vertex_normal);
    }
    
    light_source = vec3(5.0, 20.0, 10.0);
    float diffuse_red = albedo_r / 3.14 * dot(normal, light_source);
    float diffuse_green = albedo_g / 3.14 * dot(normal, light_source);
    float diffuse_blue = albedo_b / 3.14 * dot(normal, light_source);
    ambient_light = vec4(albedo_r/3.14, albedo_g/3.14, albedo_b/3.14, 0.0);

    vec3 reflection_vec = 2.0 * dot(light_source, normal) * normal - light_source;
    float specular = ks * dot(reflection_vec, normalize(camera_position));
    
    specular = max(0.0, specular);
    diffuse_red = max(0.0, diffuse_red);
    diffuse_green = max(0.0, diffuse_green);
    diffuse_blue = max(0.0, diffuse_blue);
    
    vertex_color = vec3(diffuse_red + specular, diffuse_green + specular, diffuse_blue + specular) + ambient_light.rgb;
}
'''




A **iluminação ambiente** (*ambient light*) é a luz que o objeto emite se não houver outras luzes.  

A **iluminação difusa** (*diffuse light*) é a luz que o objeto emite uniformemente em todas as direções a partir do ponto em que um raio de luz o atinge, coisas que têm uma refletância difusa alta seriam objetos com um acabamento fosco.

O parâmetro **Albedo** controla a cor base da superfície.  

In [ ]:
fragment_shader = '''

#version 330

uniform float albedo_r;
uniform float albedo_g;
uniform float albedo_b;
uniform float ks;
uniform int shading;

in vec3 normal;
in vec3 view_position;
in vec3 pixel_position;
in vec3 vertex_color;
in vec3 camera_position;

in vec3 light_source;
in vec4 ambient_light;

out vec4 frag_color;

//0 - flat shading
//1 - gouraud shading
//2 - phong shading

void main() {  
    if(shading == 0) {
        vec3 x_tangent = dFdx(pixel_position);
        vec3 y_tangent = dFdy(pixel_position);
        vec3 face_normal = normalize(cross(x_tangent, y_tangent));
        
        float red_diffuse = albedo_r / 3.14 * dot(face_normal, light_source);
        float green_diffuse = albedo_g / 3.14 * dot(face_normal, light_source);
        float blue_diffuse = albedo_b / 3.14 * dot(face_normal, light_source);
        
        vec3 reflection_vec = 2.0 * dot(light_source, face_normal) * face_normal - light_source;
        float specular = ks * dot(reflection_vec, normalize(camera_position));
        
        specular = max(0.0, specular);
        red_diffuse = max(0.0, red_diffuse);
        green_diffuse = max(0.0, green_diffuse);
        blue_diffuse = max(0.0, blue_diffuse);
        
        frag_color = vec4(red_diffuse + specular, green_diffuse + specular, blue_diffuse + specular, 1.0) + ambient_light;
    }
    else if(shading == 1){
        frag_color = vec4(vertex_color, 1.0);
    }
    else {
        vec3 interpolated_normal = normalize(normal);
        
        float red_diffuse = albedo_r / 3.14 * dot(interpolated_normal, light_source);
        float green_diffuse = albedo_g / 3.14 * dot(interpolated_normal, light_source);
        float blue_diffuse = albedo_b / 3.14 * dot(interpolated_normal, light_source);
        
        vec3 reflection_vec = 2.0 * dot(light_source, interpolated_normal) * interpolated_normal - light_source;
        float specular = ks * dot(reflection_vec, normalize(camera_position));
        
        specular = max(0.0, specular);
        red_diffuse = max(0.0, red_diffuse);
        green_diffuse = max(0.0, green_diffuse);
        blue_diffuse = max(0.0, blue_diffuse);
        
        frag_color = vec4(red_diffuse + specular, green_diffuse + specular, blue_diffuse + specular, 1.0) + ambient_light;
    }
}
'''

O tipo mais simples de shading é o *__Flat Shading__*. Ele aplica o modelo de iluminação uma única vez para
cada polígono, gerando uma intensidade que é usada para sombrea-lo.

Para produzir um resultado mais suave, pode-se utilizar o *__Gouraud Shading__*. Ele ilumina ou colora diretamente cada vértice pela utilização da sua posição e da sua normal. 

Com o OpenGL, definimos vértices e então construímos todos os objetos que queremos desenhar na tela usando triângulos. Um VBO é um array que contém dados que irão para o seu GPU.

In [18]:
def main():
    #init glfw
    if not glfw.init():
        return
    
    window = glfw.create_window(800, 600, "My Window", None, None)
    
    if not window:
        glfw.terminate()
        return
    
    glfw.make_context_current(window)
    
    # bottom r, bottom l, top 
    triangle = [-0.5, -0.5, 0.0,
                0.5, -0.5, 0.0,
                0.0, 0.5, 0.0]
    
    vertex_shader = """
    #version 330
    in vec4 position; 
    
    void main(){
        gl_Position = position;
    }
    
    """
    
    fragment_shader = """
    #version 330
    in vec4 position; 
    
    void main(){
        gl_FragColor = vec4(1.0f, 0.0f, 0.0f, 1.0f);
    }
    
    """
    
    shader = OpenGL.GL.shaders.compileProgram(OpenGL.GL.shaders.compileShader(vertex_shader, GL_VERTEX_SHADER), 
                                             OpenGL.GL.shaders.compileShader(fragment_shader, GL_FRAGMENT_SHADER))
    
    # Vertex buffer object
    VBO = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, VBO)
    glBufferData(GL_ARRAY_BUFFER, 36, triangle, GL_STATIC_DRAW)
    
    position = glGetAttribLocation(shader, "position")
    
    glClearColor(0.2, 0.8, 0.8, 1.0)
    
    while not glfw.window_should_close(window):
        glfw.poll_events()
        
        glClear(GL_COLOR_BUFFER_BIT)
        glfw.swap_buffers(window)
        
    glfw.terminate()
    
if __name__ == "__main__":
    main()

AttributeError: module 'OpenGL.GL' has no attribute 'shaders'

Create a green screen

Referências:
https://medium.com/@yvanscher/an-introduction-to-shaders-in-opengl-c19a1376eda1